In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")
print(df.head())
#dependent and independemt features
i=df.iloc[:,0:]
o=df.iloc[:,2]

from sklearn.model_selection import train_test_split
i_train,i_test,o_train,o_test=train_test_split(i,o,test_size=0.2)
#print(X_train.shape)
#print(X_test.shape)
#print(X_train.head(2))
#print(y_train.head(2))


In [ ]:
print(df["Seller_Type"].unique())
print(df["Fuel_Type"].unique())
print(df["Transmission"].unique())
print(df["Car_Name"].unique())

In [ ]:
print(df.isnull().sum())


In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
dataset=df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
dataset['current_year']=2020

In [ ]:
dataset.head()

In [ ]:
dataset['no_of_year']=dataset['current_year']-dataset['Year']
dataset.head()

In [ ]:
dataset.drop(['Year'],axis=1,inplace=True)
dataset.drop(['current_year'],axis=1,inplace=True)
dataset.head()

#  ONE HOT ENCODING TO CONVERT CATEGORICAL TO NUMERICAL 



In [ ]:
##onehot encoding

dataset=pd.get_dummies(dataset,drop_first=True)
dataset.head()

In [ ]:
dataset.corr()

In [ ]:
import seaborn as sns

In [ ]:
#sns.pairplot(dataset)

In [ ]:
#import matplotlib.pyplot as plt
#%matplotlib inline

In [ ]:
#corrmat=dataset.corr() 
#top_corr_features=corrmat.index
#plt.figure(figsize=(20,20))
#plot heat map 
#g=sns.heatmap(dataset[top_corr_features].corr(),annot=True,cmap="RdYlGn")



# FEATURE SELECTION

In [ ]:
##dependent and independemt features
X=dataset.iloc[:,1:]
Y=dataset.iloc[:,0]


In [ ]:
#feature importance
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(X,Y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of feature importances for better visualization 
feat_importances = pd.Series(model.feature_importances_, index=X.columns) 
feat_importances.nlargest(5).plot(kind='barh') 
plt.show()

# TEST TRAIN SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
print(X_train.shape)
print(X_test.shape)

# HYPERPARAMETER TRAINING IN RANDOMFORESTREGRESSOR USING RANDOMIZED SEARCH CV

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
rf_random=RandomForestRegressor()


In [ ]:
#Hyperparameters
import numpy as np
n_estimators=[int(x) for x in np.linspace(start=100,stop=1200,num=12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
#create the random grid
random_grid={'n_estimators':n_estimators,
            'max_features':max_features,
            'max_depth':max_depth,
            'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(X_test)

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

# **SUBMISSION FILE**

In [ ]:
print(predictions)
print(X_test.head())

mysubmission = pd.DataFrame({
        "Car_Name": i_test["Car_Name"],
        "Predictions": predictions
    })
mysubmission.to_csv('submission.csv', index=False)